In [ ]:
"""
pyaudio is used to capture the sound
struct is used to convert the data from binary to ints
matplot is used to graph

this version has a little lag but is usually cleaner

~20fps


look into windowing: https://www.nti-audio.com/en/support/know-how/fast-fourier-transform-fft
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
plt.close('all')

#opens new window
%matplotlib tk

#constants
Chunk = 1024*2                  #samples per frame
Format = pyaudio.paInt16        #bytes per sample
Channels = 1                    #one channel for mic
Rate = 44100                    #samples per second

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, (ax,ax2) = plt.subplots(2, figsize=(15,8))

#varible for plotting
x = np.arange(0, 2*Chunk, 2)
x_fft = np.linspace(0, Rate, Chunk)

#create a line object pulling random data - can be random because it updates in a loop so y value doesn't matter
line, = ax.plot(x, np.random.rand(Chunk), '-', lw=2)
line_fft, =ax2.semilogx(x_fft, np.random.rand(Chunk), '-', lw=2)

#basic formatting for the plot
ax.set_title('LOUDNESS')
ax.set_xlabel('sample')
ax.set_ylabel('volume')
ax.set_ylim(0,255)
ax.set_xlim(0,Chunk)

ax2.set_title('FFT')
ax2.set_xlabel('Frequency')
ax2.set_ylabel('Amplitude')
ax2.set_xlim(0, Rate / 2)

plt.tight_layout()


while True:
    #recieve binary data
    data = stream.read(Chunk, exception_on_overflow=False)
    
    #convert to int, make an array, offset by 128
    data_int = struct.unpack(str(2*Chunk) + 'B', data)
    data_np = np.array(data_int, dtype = 'b')[::2] + 128    #switching b to int removes wrap around
    
    

    line.set_ydata(data_np)
    y_fft = fft(data_int)
    y_fft = np.abs(y_fft[0:Chunk]) * 2 / (256 * Chunk) #parseval's theorem and second part of fft is always just a conjugate
    
    line_fft.set_ydata(y_fft)        
    
#     print(x_fft[np.argmax(y_fft[20:Chunk])+20])
            
    #update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()
    





/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '


In [ ]:
"""
pyaudio is used to capture the sound
struct is used to convert the data from binary to ints
matplot is used to graph

~20fps

this is hte version that uses np.frombuffer
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
plt.close('all')

#opens new window
%matplotlib tk

#constants
Chunk = 1024*2                  #samples per frame
Format = pyaudio.paInt16        #bytes per sample
Channels = 1                    #one channel for mic
Rate = 44100                    #samples per second

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, (ax,ax2) = plt.subplots(2, figsize=(15,8))

#varible for plotting
x = np.arange(0, 2*Chunk, 2)
x_fft = np.linspace(0, Rate, Chunk)
# for i in range(Chunk):
#     print(x_fft[i])

#create a line object pulling random data - can be random because it updates in a loop so y value doesn't matter
line, = ax.plot(x, np.random.rand(Chunk), '-', lw=2)
line_fft, =ax2.semilogx(x_fft, np.random.rand(Chunk), '-', lw=2)

#basic formatting for the plot
ax.set_title('LOUDNESS')
ax.set_xlabel('sample')
ax.set_ylabel('volume')
ax.set_ylim(-20000,20000)
ax.set_xlim(0,Chunk)

ax2.set_title('FFT')
ax2.set_xlabel('Frequency')
ax2.set_ylabel('Amplitude')
ax2.set_xlim(0, Rate / 2)

plt.tight_layout()


while True:
    #recieve binary data
    data = stream.read(Chunk, exception_on_overflow=False)
    
    #convert to int, make an array, offset by 128
    data_int = np.frombuffer(data, dtype = np.int16)
    data_np = np.array(data_int, dtype = np.int16) 

    line.set_ydata(data_np)
    y_fft = fft(data_int)
    y_fft = np.abs(y_fft[0:Chunk]) * 2 / (256 * Chunk) #parseval's theorem
    
    line_fft.set_ydata(y_fft)        
    
#     print(x_fft[np.argmax(y_fft[20:Chunk])+20])
            
    #update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()
    






/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '


In [ ]:
"""
pyaudio is used to capture the sound
struct is used to convert the data from binary to ints
matplot is used to graph

~20fps
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.signal import find_peaks
import time
from tkinter import TclError
plt.close('all')

#opens new window
%matplotlib tk

#constants
Chunk = 1024*2                  #samples per frame
Format = pyaudio.paInt16        #bytes per sample
Channels = 1                    #one channel for mic
Rate = 44100                    #samples per second

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, (ax,ax2) = plt.subplots(2, figsize=(15,8))

#varible for plotting
x = np.arange(0, 2*Chunk, 2)
x_fft = np.linspace(0, Rate, Chunk)

#create a line object pulling random data - can be random because it updates in a loop so y value doesn't matter
line, = ax.plot(x, np.random.rand(Chunk), '-', lw=2)
line_fft, =ax2.semilogx(x_fft, np.random.rand(Chunk), '-', lw=2)

#basic formatting for the plot
ax.set_title('LOUDNESS')
ax.set_xlabel('sample')
ax.set_ylabel('volume')
ax.set_ylim(0,255)
ax.set_xlim(0,Chunk)

ax2.set_title('FFT')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Amplitude')
ax2.set_xlim(20, Rate / 2)

plt.tight_layout()


while True:
    #recieve binary data
    data = stream.read(Chunk, exception_on_overflow=False)
    
    #convert to int, make an array, offset by 128
    data_int = struct.unpack(str(2*Chunk) + 'B', data)
    data_np = np.array(data_int, dtype = 'b')[::2] + 128    #switching b to int removes wrap around

    line.set_ydata(data_np)
    y_fft = fft(data_int)
    y_fft = np.abs(y_fft[0:Chunk]) * 2 / (256 * Chunk) #parseval's theorem
    
    line_fft.set_ydata(y_fft)
    
#     print(x_fft[np.argmax(y_fft[20:Chunk])+20])
            
    #update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()
    






In [1]:
from scipy import fft
import numpy as np

# Number of sample points
N = 600
# sample spacing
T = 1.0 / 800.0
x = np.linspace(0.0, N*T, N, endpoint=False)
y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
yf = fft(y)

print(x,y)

import matplotlib.pyplot as plt

plt.plot(x, yf)
plt.plot(x, y)

plt.grid()
plt.show()

[0.      0.00125 0.0025  0.00375 0.005   0.00625 0.0075  0.00875 0.01
 0.01125 0.0125  0.01375 0.015   0.01625 0.0175  0.01875 0.02    0.02125
 0.0225  0.02375 0.025   0.02625 0.0275  0.02875 0.03    0.03125 0.0325
 0.03375 0.035   0.03625 0.0375  0.03875 0.04    0.04125 0.0425  0.04375
 0.045   0.04625 0.0475  0.04875 0.05    0.05125 0.0525  0.05375 0.055
 0.05625 0.0575  0.05875 0.06    0.06125 0.0625  0.06375 0.065   0.06625
 0.0675  0.06875 0.07    0.07125 0.0725  0.07375 0.075   0.07625 0.0775
 0.07875 0.08    0.08125 0.0825  0.08375 0.085   0.08625 0.0875  0.08875
 0.09    0.09125 0.0925  0.09375 0.095   0.09625 0.0975  0.09875 0.1
 0.10125 0.1025  0.10375 0.105   0.10625 0.1075  0.10875 0.11    0.11125
 0.1125  0.11375 0.115   0.11625 0.1175  0.11875 0.12    0.12125 0.1225
 0.12375 0.125   0.12625 0.1275  0.12875 0.13    0.13125 0.1325  0.13375
 0.135   0.13625 0.1375  0.13875 0.14    0.14125 0.1425  0.14375 0.145
 0.14625 0.1475  0.14875 0.15    0.15125 0.1525  0.15375 0.155   

/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


<Figure size 640x480 with 1 Axes>

In [ ]:
"""
Compute the discrete Fourier Transform of the 1D array x
:param x: (array)
"""
import numpy as np

N = x.size
n = np.arange(N)
k = n.reshape((N, 1))
e = np.exp(-2j * np.pi * k * n / N)
return np.dot(e, x)